In [1]:
import numpy as np
import pandas as pd
import re
from convokit import Corpus, download
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score

In [2]:
utts = pd.read_csv('data/utts.csv')
roberts = pd.read_csv('data/subset.csv')
#UPDATE THIS WHOLE FILE TO LOAD IN THE UNIGRAM BAG OF WORDS FEATURE SETS AND WORK FROM THERE

In [3]:
"""
Everything below is for a Naive Bayes model which is distinctly 
not what we said but I wanted to get a general feel for building a model and things that we may want to tweak 
for our Bag of Words and beyond.
"""
utts['text'].replace('\d+', '', regex=True, inplace = True) #remove numbers, decide as group if this makes sense
group_utts = utts.groupby('meta.case_id')['text'].apply(' '.join)
#NOTE: No stemming or lemmatization done at this point
df = pd.merge(group_utts,roberts[['id','win_side']], how = 'left', 
              left_on = 'meta.case_id', right_on = 'id')
X_train, X_test, y_train, y_test = train_test_split(
    df.loc[:,df.columns != 'win_side'],df['win_side'], test_size = 0.2, random_state = 0)

In [4]:
count_vect = CountVectorizer(stop_words = 'english') #play around with ngrams, max/min df args
count_df = count_vect.fit_transform(X_train['text'])
count_array = count_df.toarray()
count_df = pd.DataFrame(count_array,columns = count_vect.get_feature_names(), index = X_train['id'])

In [5]:
count_test = count_vect.transform(X_test['text'])
count_test_array = count_test.toarray()
count_test_df = pd.DataFrame(count_test_array, columns = count_vect.get_feature_names(),index = X_test['id'])

In [6]:
model = GaussianNB() 
model.fit(count_df,y_train)
win_pred = model.predict(count_test_df)

print(accuracy_score(y_test,win_pred))
"""
this accuracy is notably worse than the 'just say the petitioner will win 
every time and you'll be right 65% of the time'
"""
print(precision_score(y_test,win_pred))

0.5902439024390244
0.627906976744186
